## Evaluate Model

## Load trained model

In [1]:
import sys
# import the py file for loading the dataset
if "..\\121-layer\\src" not in sys.path:
    sys.path.insert(0,r'..\121-layer\src')
print(sys.path)

['..\\121-layer\\src', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\notebooks', 'C:\\Python312\\python312.zip', 'C:\\Python312\\DLLs', 'C:\\Python312\\Lib', 'C:\\Python312', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv', '', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages', 'C:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\Pythonwin']


In [2]:
import torch
torch.manual_seed(42)

from custom_densenet import *
from preprocessing import *
train_dataset, val_dataset,train_loader, val_loader,test_dataset, test_loader= get_data_loaders(data_dir='../raw_data/archive/', label_file='../raw_data/archive/CXR8-selected/Data_Entry_2017_v2020.csv')

Training Dataset Size: 8484
Validation Dataset Size: 1060
Test Dataset Size: 1060


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
pathModel = r"..\notebooks\dnet_models\m-custom_dnet_binary_by_img_count_lr_1e-4.pth.tar"

model = dense_net(1, training=False)
modelCheckpoint = torch.load(pathModel)
model.load_state_dict(modelCheckpoint['state_dict'])
model.to(device)

dense_net(
  (initial_setup): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (denseblock1): dense_block(
    (dense_layer0): dense_layer(
      (net): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
        (5): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
    (dense_layer1): dense_layer(
      (net): Sequential(
        (0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

## Evaluate model on the test_dataset

In [4]:
CLASS_NAMES = ['Pneumothorax']
outGT = torch.FloatTensor().cuda()
outPRED = torch.FloatTensor().cuda()

tp = [0 for i in range(1,10)]
fp = [0 for i in range(1,10)]
fn = [0 for i in range(1,10)]

def f1_score(tp, fp, fn):
    return 2 * (tp) / (2 * tp + fp + fn)

model.eval()
for i, (input, target) in enumerate(test_loader):
    input = input.to(device)
    target = target.cuda()
    outGT = torch.cat((outGT, target), 0)
    
    with torch.no_grad():    
        out = model(input)
        
        for i in range(1,10):
            pred_labels = (out > i/10).long().T.squeeze(0)
            tp[i-1] += sum(torch.logical_and(pred_labels, target))
            fp[i-1] += sum(torch.logical_and(torch.logical_xor(pred_labels, target).long(), pred_labels))
            fn[i-1] += sum(torch.logical_and(torch.logical_xor(pred_labels, target).long(), target))

        outPRED = torch.cat((out, out.data), 0)

for i in range(1,10):
            print(f"confidence threshold {i/10}")
            print("tp_sum: {:.4f}, fp_sum: {:.4f}, fn_sum: {:.4f}, cumulative_f1_score: {:.4f}".format(
                                                                     tp[i-1], \
                                                                     fp[i-1],\
                                                                     fn[i-1],\
                                                                     f1_score(tp[i-1], fp[i-1], fn[i-1])))
        
     
# aurocIndividual = computeAUROC(outGT, outPRED, nnClassCount)
# aurocMean = np.array(aurocIndividual).mean()

# print ('AUROC mean ', aurocMean)

# for i in range (0, len(aurocIndividual)):
#     print (CLASS_NAMES[i], ' ', aurocIndividual[i])


confidence threshold 0.1
tp_sum: 521.0000, fp_sum: 539.0000, fn_sum: 0.0000, cumulative_f1_score: 0.6591
confidence threshold 0.2
tp_sum: 519.0000, fp_sum: 538.0000, fn_sum: 2.0000, cumulative_f1_score: 0.6578
confidence threshold 0.3
tp_sum: 507.0000, fp_sum: 515.0000, fn_sum: 14.0000, cumulative_f1_score: 0.6572
confidence threshold 0.4
tp_sum: 462.0000, fp_sum: 435.0000, fn_sum: 59.0000, cumulative_f1_score: 0.6516
confidence threshold 0.5
tp_sum: 375.0000, fp_sum: 326.0000, fn_sum: 146.0000, cumulative_f1_score: 0.6137
confidence threshold 0.6
tp_sum: 170.0000, fp_sum: 104.0000, fn_sum: 351.0000, cumulative_f1_score: 0.4277
confidence threshold 0.7
tp_sum: 17.0000, fp_sum: 11.0000, fn_sum: 504.0000, cumulative_f1_score: 0.0619
confidence threshold 0.8
tp_sum: 5.0000, fp_sum: 1.0000, fn_sum: 516.0000, cumulative_f1_score: 0.0190
confidence threshold 0.9
tp_sum: 1.0000, fp_sum: 0.0000, fn_sum: 520.0000, cumulative_f1_score: 0.0038
